# **IMPORT**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings, OpenAIEmbeddings
from langchain.llms import CTransformers
from langchain.chains.llm import LLMChain
from langchain_community.graphs import Neo4jGraph
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains import RetrievalQA, GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

from neo4j import GraphDatabase

import numpy as np
import pandas as pd
import getpass
import re

import json
import os
import pickle

# **FUNCTION**

In [1]:
def load_pickle(path):
    with open(path, 'rb') as file:
        data = pickle.load(file)
    return data


In [2]:
# Preprocessing data
def clean_text(
        text,
        methods=['rmv_link', 'rmv_punc', 'lower', 'replace_word', 'rmv_space'],
        custom_punctuation = '!"#$%&\'()*+,-:;<=>?@[\\]^_`{|}~”“',
        patterns=[],
        words_replace=[],
        rdrsegmenter=None,
    ):
    cleaned_text = text
    for method in methods:
        if method == 'rmv_link':
            # Remove link
            cleaned_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned_text)
            cleaned_text = "".join(cleaned_text)
        elif method == 'rmv_punc':
            # Remove punctuation
            cleaned_text = re.sub('[%s]' % re.escape(custom_punctuation), '' , cleaned_text)
        elif method == 'lower':
            # Lowercase
            cleaned_text = cleaned_text.lower()
        elif method == 'replace_word':
            # Replace word
            for pattern, repl in zip(patterns, words_replace):
                cleaned_text = re.sub(pattern, repl, cleaned_text)
        elif method == 'rmv_space':
            # Remove extra space
            cleaned_text = re.sub(' +', ' ', cleaned_text)
            cleaned_text = cleaned_text.strip()
        elif method == 'segmentation':
            if rdrsegmenter is None:
                print('No Segmenter found !!')
                continue
            # Word and cleaned_text segmentation
            cleaned_text = rdrsegmenter.word_segment(cleaned_text)
            cleaned_text = ' '.join(cleaned_text)

    return cleaned_text

<>:14: SyntaxWarning: invalid escape sequence '\('
<>:14: SyntaxWarning: invalid escape sequence '\('
C:\Users\ASUS\AppData\Local\Temp\ipykernel_6948\1992846485.py:14: SyntaxWarning: invalid escape sequence '\('
  cleaned_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned_text)


# **LOAD DATA**